In [15]:

import pandas as pd
import numpy as np
import torch
from sklearn.preprocessing import StandardScaler as ss
from sklearn.utils import shuffle


In [16]:
data=pd.read_csv("./csv_building_damage_assessment.csv",na_values=["None",",","nan","\n"])
data=data.drop(["building_id","district_id","vdcmun_id","ward_id"],axis=1)
for columns in data.columns:
    if data[columns].count()<200000:
        data=data.drop(columns,axis=1)

data=data.dropna()


/home/student/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,17,29,34,35,36,37,38,39,40,41,42,43,47,50) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [17]:
features=pd.get_dummies(data)
labels=data["damage_grade"]

un_labels=labels.unique()
labels=pd.get_dummies(labels)


In [18]:
features=np.array(features)
labels=np.array(labels)
features=ss().fit_transform(features)
features=torch.tensor(features,dtype=torch.float32)
labels=torch.tensor(labels,dtype=torch.float32)


print(features.shape,labels.shape)

torch.Size([20201, 40]) torch.Size([20201, 5])


In [19]:
from torch.utils.data import TensorDataset,DataLoader
BATCH_SIZE=1000

dataset=TensorDataset(features[:199000],labels[:199000])
dataloader=DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True)


In [21]:
model=torch.nn.Sequential(
    torch.nn.Linear(40,80),
    torch.nn.ReLU(),
    torch.nn.Linear(80,30),
    torch.nn.ReLU(),
    torch.nn.Linear(30,15),
    torch.nn.ReLU(),
    torch.nn.Linear(15,5),
    torch.nn.Softmax(dim=-1)
)

In [22]:
loss_fn=torch.nn.BCELoss()

In [23]:
opt=torch.optim.Adam(model.parameters(),lr=1e-4)

In [24]:
from torch import no_grad
epochs=1000
for epoch in range(epochs):
    for features,labels in dataloader:
        preds=model(features)
        loss=loss_fn(preds,labels)

        loss.backward()
        opt.step()
        opt.zero_grad()
  
    with torch.no_grad():
        if (epoch+1)%(epochs//10)==0:
            right=0
            for features,labels in dataloader:
                preds=model(features)
                for i in range(len(preds)):
                    for j in range(len(preds[i])):

                        if preds[i][j]==max(preds[i]):
                            if(labels[i][j]==1):
                                right+=1

            print("Accuracy:",round(right*100/(len(dataloader)*BATCH_SIZE),2))                   

Accuracy: 96.18095
Accuracy: 96.19524
Accuracy: 96.19524
Accuracy: 96.19524
Accuracy: 96.19524
Accuracy: 96.19524
Accuracy: 96.19524
Accuracy: 96.19524
Accuracy: 96.19524
Accuracy: 96.19524


In [25]:
torch.save(model.state_dict(),"earthquakedamage.pth")